# Optimization for Large-Scale Data (2025-2026)
## First Assignment: Linear Optimization Modeling

* **Student Name:** Cano Turnes, Pablo
* **Student Number:** 100558677

---

### 1. Problem Definition: Multi-Period Hospital Resource Allocation

**Context:**
As the newly appointed Director of Operations Research for a major Public Hospital, I am tasked with improving resource utilization under limited budgets and high patient demand.

**Objective:**
The goal is to optimize the allocation of critical resources (Doctors, Nurses, Maintenance, and Administrative staff) across various hospital departments (Emergency Room, ICU, Surgery, Hospital Floors, and Outpatient Clinics) over a planning horizon of a full week. The objective is to maximize the "Quality Adjusted Service Level," a metric derived from the priority of the department and the volume of resources deployed, while adhering to strict supply, demand, physical capacity, and budget constraints.

**Problem Dimensions:**
To ensure the model is robust and meets the assignment requirements of at least 100 decision variables and 20 constraints, the problem is defined as follows:
* **5 Departments** ($D$)
* **4 Resource Types** ($R$)
* **12 Shifts** ($S$) (Mon-Fri AM/PM + Weekend AM/PM)

This results in $5 \times 4 \times 12 = 240$ continuous decision variables.

---

### a) Mathematical Formulation (Linear Optimization Model)

Following the general formulation of an LP, the problem is defined by the following sets, parameters, variables, and constraints.

#### 1. Sets
* $D = \{ER, ICU, Surgery, HospitalFloors, OutpatientClinics\}$: Set of hospital departments ($|D|=5$).
* $R = \{Doctors, Nurses, Maintenance, Administrative\}$: Set of resource types ($|R|=4$).
* $S = \{MonAM, MonPM, \dots, FriPM, WeekendAM, WeekendPM\}$: Set of shifts ($|S|=12$).

#### 2. Parameters
* $P_{d}$: Priority weight of department $d$ (reflecting urgency, e.g., ER has highest priority).
* $U_{r}$: Utility factor of resource $r$ (Clinical contribution per unit).
* $C_{r}$: Cost per unit of resource $r$ (in €).
* $L_{r,s}$: Total availability limit of resource $r$ during shift $s$ (Supply).
* $M_{d,r}$: Minimum required units of resource $r$ in department $d$ to maintain basic operations (Demand).
* $Cap_{d,r}$: Maximum physical capacity of department $d$ for resource $r$ (Space/Equipment limits).
* $B$: Total operational budget for the planning horizon.

#### 3. Decision Variables
* $x_{d,r,s} \in \mathbb{R}_{\ge 0}$: The quantity of resource $r$ (in units) allocated to department $d$ during shift $s$.

#### 4. Objective Function
Maximize the total weighted service quality provided across all departments and shifts:

$$
\max Z = \sum_{d \in D} \sum_{r \in R} \sum_{s \in S} (P_{d} \cdot U_{r}) \cdot x_{d,r,s}
$$

The objective function is designed to maximize the total "Quality Adjusted Service Level" rather than strictly minimizing costs, reflecting the public sector mandate to prioritize health outcomes over profit. By integrating a Department Priority ($P_d$) and a Resource Utility factor ($U_r$), the model ensures that scarce resources are preferentially allocated to high-urgency areas like the ER or ICU and properly values the higher clinical contribution of specialized staff compared to basic equipment. This weighted approach creates a robust quantitative proxy for patient care quality, guiding the solver toward the most clinically efficient distribution of resources while strictly adhering to the defined budgetary and physical constraints.

#### 5. Constraints

**i. Resource Availability (Supply Constraints):**
The total resources allocated across all departments in a specific shift cannot exceed the available supply for that shift.
$$
\sum_{d \in D} x_{d,r,s} \le L_{r,s} \quad \forall r \in R, \forall s \in S
$$

**ii. Minimum Service Standards (Demand Constraints):**
Every department must receive at least the minimum required resources to function safely.
$$
x_{d,r,s} \ge M_{d,r} \quad \forall d \in D, \forall r \in R, \forall s \in S
$$

**iii. Maximum Physical Capacity (Diminishing Returns):**
To prevent unrealistic "hoarding" of resources by high-priority departments (e.g., placing 100 doctors in an ER that only has physical space for 20), we impose an upper bound.
$$
x_{d,r,s} \le Cap_{d,r} \quad \forall d \in D, \forall r \in R, \forall s \in S
$$

**iv. Budget Constraint:**
The total cost of allocated resources must not exceed the global budget.
$$
\sum_{d \in D} \sum_{r \in R} \sum_{s \in S} C_{r} \cdot x_{d,r,s} \le B
$$

**v. Non-negativity:**
$$
x_{d,r,s} \ge 0
$$

In [134]:
import pyomo.environ as pyo
import random
import pandas as pd

# 1. ROBUST DATA GENERATION (Full Week)
# ---------------------------------------------------------
depts = ['ER', 'ICU', 'Surgery', 'Hospital_floors', 'Outpatient_Clinics']
resources = ['Doctors', 'Nurses', 'Maintenance', 'Administrative']
shifts = [
    'MonAM', 'MonPM', 'TueAM', 'TuePM', 'WedAM', 'WedPM',
    'ThuAM', 'ThuPM', 'FriAM', 'FriPM', 'weekendAM', 'weekendPM'
]

# Parameters
random.seed(42)

# Priorities & Utility & Cost
priority_map = {'ER': 1, 'ICU': 0.75, 'Surgery': 0.5, 'Hospital_floors': 0.25, 'Outpatient_Clinics': 0.1}
P = {d: priority_map[d] for d in depts}
U = {'Doctors': 1, 'Nurses': 0.5, 'Maintenance': 0.2, 'Administrative': 0.1}
C = {'Doctors': 200, 'Nurses': 80, 'Maintenance': 30, 'Administrative': 40}

# Supply (L) and Demand (M) - Using the "Real World" logic (High AM / Low PM)
L = {} 
M = {}
# Minimum demands
M['ER', 'Doctors'] = 10; M['ER', 'Nurses'] = 30; M['ER', 'Administrative'] = 3; M['ER', 'Maintenance'] = 2
M['ICU', 'Doctors'] = 5; M['ICU', 'Nurses'] = 15; M['ICU', 'Administrative'] = 2; M['ICU', 'Maintenance'] = 2
M['Surgery', 'Doctors'] = 20; M['Surgery', 'Nurses'] = 60; M['Surgery', 'Administrative'] = 4; M['Surgery', 'Maintenance'] = 2
M['Hospital_floors', 'Doctors'] = 40; M['Hospital_floors', 'Nurses'] = 120; M['Hospital_floors', 'Administrative'] = 10; M['Hospital_floors', 'Maintenance'] = 5
M['Outpatient_Clinics', 'Doctors'] = 100; M['Outpatient_Clinics', 'Nurses'] = 105; M['Outpatient_Clinics', 'Administrative'] = 50; M['Outpatient_Clinics', 'Maintenance'] = 5

# Auto-filling missing M if any, and generating L
min_total_req = {r: sum(M.get((d,r), 1) for d in depts) for r in resources}

for r in resources:
    base_low = int(min_total_req[r] * 1.1) # 10% buffer
    base_high = base_low * 10
    for s in shifts:
        is_high_demand = ('AM' in s) and ('weekend' not in s)
        if is_high_demand:
             L[r, s] = int(base_high * random.uniform(0.95, 1.05))
        else:
             L[r, s] = int(base_low * random.uniform(1.0, 1.1))

# Generating Max Capacity based on Min Demand
Max = {}
for d in depts:
    for r in resources:
        # Give High Priority depts more room to grow, but cap low priority ones strictly
        # This prevents the solver from dumping 1000 doctors into the ER
        limit_factor = 2.0 if d in ['ER', 'ICU'] else 1.2
        # Ensure Max is at least equal to Min to avoid infeasibility
        Max[d, r] = int(M.get((d,r), 1) * limit_factor)

# Adjusted Budget for full scale
Budget = 900000 

# 2. MODEL
# ---------------------------------------------------------
model = pyo.ConcreteModel(name="Hospital_Resource_Allocation_Final")
model.D = pyo.Set(initialize=depts)
model.R = pyo.Set(initialize=resources)
model.S = pyo.Set(initialize=shifts)
model.x = pyo.Var(model.D, model.R, model.S, domain=pyo.NonNegativeReals)

# Obj
def obj_rule(m):
    return sum(P[d] * U[r] * m.x[d,r,s] for d in m.D for r in m.R for s in m.S)
model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
model.SupplyConstr = pyo.Constraint(model.R, model.S, rule=lambda m, r, s: sum(m.x[d,r,s] for d in m.D) <= L[r,s])
model.DemandConstr = pyo.Constraint(model.D, model.R, model.S, rule=lambda m, d, r, s: m.x[d,r,s] >= M.get((d,r), 1))
model.BudgetConstr = pyo.Constraint(rule=lambda m: sum(C[r] * m.x[d,r,s] for d in m.D for r in m.R for s in m.S) <= Budget)

# --- NEW: Capacity Constraint ---
def capacity_rule(m, d, r, s):
    return m.x[d,r,s] <= Max[d,r]
model.CapacityConstr = pyo.Constraint(model.D, model.R, model.S, rule=capacity_rule)

# Solve
solver = pyo.SolverFactory('glpk') 
results = solver.solve(model)

print(f"Status: {results.solver.status}")
print(f"Objective score: {pyo.value(model.Obj):.2f}")

Status: ok
Objective score: 1719.37


In [135]:
import pandas as pd

# Extracting results
allocation_data = []
for d in depts:
    for r in resources:
        for s in shifts:
            allocated_amount = pyo.value(model.x[d, r, s])
            allocation_data.append({
                'Department': d,
                'Resource': r,
                'Shift': s,
                'Allocated_Units': allocated_amount,
                'Priority': P[d],
                'Min_Required': M[d, r],
                'Supply_Limit': L[r, s]
            })

df_allocations = pd.DataFrame(allocation_data)

# 1. Calculate Costs
df_allocations['Cost_Entry'] = df_allocations.apply(
    lambda row: row['Allocated_Units'] * C[row['Resource']], axis=1
)

# 2. Pivot & Transpose
# We pivot normally first
pivot_df = df_allocations.pivot_table(
    index=['Department', 'Resource'], 
    columns='Shift', 
    values='Allocated_Units', 
    aggfunc='sum'
)
pivot_df = pivot_df.reindex(columns=shifts)

# 3. Add Totals (before transposing, while it's easier to sum rows/cols)
pivot_df['Total Units'] = pivot_df.sum(axis=1)

money_per_shift = df_allocations.groupby('Shift')['Cost_Entry'].sum().reindex(shifts)
total_money_spent = money_per_shift.sum()
bottom_row_values = money_per_shift.tolist() + [total_money_spent]
pivot_df.loc[('TOTALS', 'Money Spent (€)'), :] = bottom_row_values

# --- TRANSPOSE THE MAIN TABLE HERE ---
pivot_df = pivot_df.T

# 4. Styling
def style_clean(df):
    return df.style.format("{:.2f}") \
        .set_table_styles([
            {'selector': 'th', 'props': [('border', '1px solid #555'), ('padding', '8px')]},
            {'selector': 'td', 'props': [('border', '1px solid #555'), ('padding', '8px')]},
        ]) 

print("Allocation Table:")
display(style_clean(pivot_df))


print("\n--- GENERATED PARAMETERS ---")

# 1. Supply Limits (Transposed for readability: Shift x Resource)
print("\nSupply Limits (L_r,s):")
supply_data = [{'Resource': r, 'Shift': s, 'Limit': L[r,s]} for r in resources for s in shifts]
df_supply = pd.DataFrame(supply_data).pivot(index='Shift', columns='Resource', values='Limit') # Note index/columns swapped
df_supply = df_supply.reindex(index=shifts)
display(df_supply)

# 2. Minimum Demand (M)
print("\nMinimum Demand Requirements (M_d,r):")
demand_data = [{'Department': d, 'Resource': r, 'Min': M[d,r]} for d in depts for r in resources]
df_demand = pd.DataFrame(demand_data).pivot(index='Department', columns='Resource', values='Min')
display(df_demand)

Allocation Table:



--- GENERATED PARAMETERS ---

Supply Limits (L_r,s):


Resource,Administrative,Doctors,Maintenance,Nurses
Shift,,,,
MonAM,774,1946,177,3458
MonPM,79,192,17,370
TueAM,777,1876,163,3684
TuePM,79,196,17,382
WedAM,765,1965,175,3528
WedPM,75,204,18,384
ThuAM,729,1995,175,3742
ThuPM,77,193,18,363
FriAM,718,1905,170,3741



Minimum Demand Requirements (M_d,r):


Resource,Administrative,Doctors,Maintenance,Nurses
Department,,,,
ER,3,10,2,30
Hospital_floors,10,40,5,120
ICU,2,5,2,15
Outpatient_Clinics,50,100,5,105
Surgery,4,20,2,60


In [136]:
# ---------------------------------------------------------
# TASK C: SENSITIVITY ANALYSIS (Shadow Prices)
# ---------------------------------------------------------

# 1. Prepare Model for Duals
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# 2. Re-Solve
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

print("Sensitivity Analysis: Identifying Bottlenecks")
print("-" * 50)

# 3. Extract Shadow Prices for Supply Constraints
supply_duals = []
for r in resources:
    for s in shifts:
        # Get the dual value for the constraint: sum(x) <= L[r,s]
        shadow_price = model.dual[model.SupplyConstr[r,s]]
        
        # Only keep non-zero values (Active Constraints)
        if abs(shadow_price) > 0.001:
            supply_duals.append({
                'Resource': r,
                'Shift': s,
                'Shadow_Price': shadow_price,
                'Current_Limit': L[r,s]
            })

# Display Supply Bottlenecks
if supply_duals:
    df_duals = pd.DataFrame(supply_duals).sort_values(by='Shadow_Price', ascending=False)
    print("Critical Resource Shortages (Top Bottlenecks):")
    print("Interpretation: Gaining 1 more unit of these resources increases Total Quality by 'Shadow_Price'.")
    display(df_duals.head(10))
else:
    print("\nNo specific resource shortages found (Supply constraints might be loose).")

# 4. Extract Shadow Price for Budget
budget_shadow_price = model.dual[model.BudgetConstr]

print("Budget Sensitivity:")
if abs(budget_shadow_price) > 0.0001:
    print(f"Shadow Price: {budget_shadow_price:.4f}")
    print(f"Interpretation: For every extra €1 invested, the Service Quality increases by {budget_shadow_price:.4f} points.")
    print("The Budget is fully utilized (Binding Constraint).")
else:
    print("Shadow Price: 0.0")
    print("Interpretation: You have leftover budget. Adding money won't improve quality because other constraints (like Capacity or Supply) are stopping you first.")

# 5. Extract Capacity Bottlenecks (Physical Space)
cap_duals = []
for d in depts:
    for r in resources:
        for s in shifts:
             shadow_price = model.dual[model.CapacityConstr[d,r,s]]
             if abs(shadow_price) > 0.001:
                 cap_duals.append({
                     'Dept': d,
                     'Resource': r,
                     'Shift': s,
                     'Shadow_Price': shadow_price
                 })

if cap_duals:
    print("\n🏥 Physical Capacity Constraints Hit:")
    print("These departments are maxed out physically.")
    df_cap = pd.DataFrame(cap_duals).sort_values(by='Shadow_Price', ascending=False)
    display(df_cap.head(10))

Sensitivity Analysis: Identifying Bottlenecks
--------------------------------------------------
Critical Resource Shortages (Top Bottlenecks):
Interpretation: Gaining 1 more unit of these resources increases Total Quality by 'Shadow_Price'.


,Resource,Shift,Shadow_Price,Current_Limit
0,Doctors,MonPM,0.37500,192
2,Doctors,ThuPM,0.37500,193
3,Doctors,FriPM,0.37500,192
12,Nurses,weekendPM,0.32500,368
6,Nurses,MonPM,0.32500,370
9,Nurses,ThuPM,0.32500,363
7,Nurses,TuePM,0.20000,382
8,Nurses,WedPM,0.20000,384
11,Nurses,weekendAM,0.20000,375
18,Maintenance,weekendAM,0.18125,17


Budget Sensitivity:
Shadow Price: 0.0006
Interpretation: For every extra €1 invested, the Service Quality increases by 0.0006 points.
The Budget is fully utilized (Binding Constraint).

🏥 Physical Capacity Constraints Hit:
These departments are maxed out physically.


,Dept,Resource,Shift,Shadow_Price
0,ER,Doctors,MonAM,0.875
8,ER,Doctors,FriAM,0.875
2,ER,Doctors,TueAM,0.875
4,ER,Doctors,WedAM,0.875
5,ER,Doctors,WedPM,0.875
6,ER,Doctors,ThuAM,0.875
11,ER,Doctors,weekendPM,0.750
10,ER,Doctors,weekendAM,0.750
3,ER,Doctors,TuePM,0.750
44,ICU,Doctors,MonAM,0.625


### d) The Dual Problem

We derive the Dual formulation to verify the robustness of our model and check for Strong Duality.

**1. Dual Variables**
We define a dual variable for every constraint in the Primal problem:
* $\lambda_{r,s} \ge 0$: Shadow price of Supply constraint ($L_{r,s}$).
* $\mu_{d,r,s} \ge 0$: Shadow price of Minimum Demand constraint ($M_{d,r}$). Note: Since the primal constraint was $\ge$, we treat this variable's contribution as negative in the algebraic sum or invert the inequality.
* $\theta_{d,r,s} \ge 0$: Shadow price of Capacity constraint ($Cap_{d,r}$).
* $\delta \ge 0$: Shadow price of the Budget constraint ($B$).

**2. Dual Objective Function**
Minimize the total valuation of the available resources:
$$
\min W = \sum_{r \in R}\sum_{s \in S} L_{r,s}\lambda_{r,s} + \sum_{d \in D}\sum_{r \in R}\sum_{s \in S} Cap_{d,r}\theta_{d,r,s} + B\delta - \sum_{d \in D}\sum_{r \in R}\sum_{s \in S} M_{d,r}\mu_{d,r,s}
$$

**3. Dual Constraints**
There is one constraint for each decision variable $x_{d,r,s}$ in the primal:
$$
\lambda_{r,s} + \theta_{d,r,s} + C_r \cdot \delta - \mu_{d,r,s} \ge P_d \cdot U_r \quad \forall d \in D, \forall r \in R, \forall s \in S
$$

**4. Strong Duality Theorem**
If the primal problem has an optimal solution $Z^*$, the dual problem also has an optimal solution $W^*$, and $Z^* = W^*$.

In [137]:
# ---------------------------------------------------------
# TASK D: SOLVING THE DUAL PROBLEM
# ---------------------------------------------------------

# Create Dual Model
dual_model = pyo.ConcreteModel(name="Dual_Hospital_Allocation")

# Sets (Same as Primal)
dual_model.D = pyo.Set(initialize=depts)
dual_model.R = pyo.Set(initialize=resources)
dual_model.S = pyo.Set(initialize=shifts)

# Dual Variables
# lambda: Supply (indexed by Resource, Shift)
dual_model.lam = pyo.Var(dual_model.R, dual_model.S, domain=pyo.NonNegativeReals)

# theta: Capacity (indexed by Dept, Resource, Shift) (Note: reusing indices of x)
dual_model.theta = pyo.Var(dual_model.D, dual_model.R, dual_model.S, domain=pyo.NonNegativeReals)

# delta: Budget (Scalar)
dual_model.delta = pyo.Var(domain=pyo.NonNegativeReals)

# mu: Demand (indexed by Dept, Resource, Shift)
dual_model.mu = pyo.Var(dual_model.D, dual_model.R, dual_model.S, domain=pyo.NonNegativeReals)

# Dual Objective Function (Minimize)
def dual_obj_rule(m):
    term_supply = sum(L[r,s] * m.lam[r,s] for r in m.R for s in m.S)
    term_cap = sum(Max[d,r] * m.theta[d,r,s] for d in m.D for r in m.R for s in m.S)
    term_budget = Budget * m.delta
    term_demand = sum(M.get((d,r), 0) * m.mu[d,r,s] for d in m.D for r in m.R for s in m.S)
    
    return term_supply + term_cap + term_budget - term_demand

dual_model.DualObj = pyo.Objective(rule=dual_obj_rule, sense=pyo.minimize)

# Dual Constraints
# One constraint per Primal Variable x[d,r,s]
# Equation: lam + theta + C*delta - mu >= P*U
def dual_constraint_rule(m, d, r, s):
    lhs = m.lam[r,s] + m.theta[d,r,s] + (C[r] * m.delta) - m.mu[d,r,s]
    rhs = P[d] * U[r]
    return lhs >= rhs

dual_model.DualConstr = pyo.Constraint(dual_model.D, dual_model.R, dual_model.S, rule=dual_constraint_rule)

# Solve Dual
results_dual = solver.solve(dual_model)

# ---------------------------------------------------------
# VERIFICATION (Strong Duality)
# ---------------------------------------------------------
primal_val = pyo.value(model.Obj)
dual_val = pyo.value(dual_model.DualObj)

print(f"Primal Objective Value (Z): {primal_val:.4f}")
print(f"Dual Objective Value (W):   {dual_val:.4f}")
print(f"Difference: {abs(primal_val - dual_val):.6f}")

if abs(primal_val - dual_val) < 0.1:
    print("\n✅ STRONG DUALITY HOLDS. The model is mathematically consistent.")
else:
    print("\n❌ Warning: Gap detected. Check formulation.")

Primal Objective Value (Z): 1719.3750
Dual Objective Value (W):   1719.3750
Difference: 0.000000

✅ STRONG DUALITY HOLDS. The model is mathematically consistent.


In [138]:
# ---------------------------------------------------------
# TASK E: INTEGER OPTIMIZATION MODEL
# ---------------------------------------------------------

# 1. Create Integer Model
# We clone the structure but change the domain to NonNegativeIntegers
int_model = pyo.ConcreteModel(name="Hospital_Resource_Allocation_Integer")

int_model.D = pyo.Set(initialize=depts)
int_model.R = pyo.Set(initialize=resources)
int_model.S = pyo.Set(initialize=shifts)

# Domain is now NonNegativeIntegers
int_model.x = pyo.Var(int_model.D, int_model.R, int_model.S, domain=pyo.NonNegativeIntegers)

# Objective
int_model.Obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints (Same rules)
int_model.SupplyConstr = pyo.Constraint(int_model.R, int_model.S, rule=lambda m, r, s: sum(m.x[d,r,s] for d in m.D) <= L[r,s])
int_model.DemandConstr = pyo.Constraint(int_model.D, int_model.R, int_model.S, rule=lambda m, d, r, s: m.x[d,r,s] >= M.get((d,r), 1))
int_model.BudgetConstr = pyo.Constraint(rule=lambda m: sum(C[r] * m.x[d,r,s] for d in m.D for r in m.R for s in m.S) <= Budget)
int_model.CapacityConstr = pyo.Constraint(int_model.D, int_model.R, int_model.S, rule=capacity_rule)

# Solve Integer Model
print("Solving Integer Model...")
solver = pyo.SolverFactory('glpk')
results_int = solver.solve(int_model)

# ---------------------------------------------------------
# TASK F: COMPARISON (RELAXED VS INTEGER)
# ---------------------------------------------------------

# Get Objective Values
obj_relaxed = pyo.value(model.Obj)      # From your previous Continuous run
obj_integer = pyo.value(int_model.Obj)  # From this Integer run

print("\n--- TASK F: Comparison of Solutions ---")
print(f"Relaxed (Continuous) Objective: {obj_relaxed:.2f}")
print(f"Integer (Realistic) Objective:  {obj_integer:.2f}")
print(f"Gap (Integrality Loss):         {obj_relaxed - obj_integer:.2f}")

if obj_relaxed >= obj_integer:
    print("\nObservation: The Relaxed solution provides an UPPER BOUND to the Integer solution.")
    print("The gap represents the 'loss' in theoretical efficiency due to the reality")
    print("that we cannot split resources (e.g., hiring 0.7 nurses).")

Solving Integer Model...

--- TASK F: Comparison of Solutions ---
Relaxed (Continuous) Objective: 1719.37
Integer (Realistic) Objective:  1719.37
Gap (Integrality Loss):         0.00

Observation: The Relaxed solution provides an UPPER BOUND to the Integer solution.
The gap represents the 'loss' in theoretical efficiency due to the reality
that we cannot split resources (e.g., hiring 0.7 nurses).
